# 03 - Detección de Cambios
## Laboratorio 2: Detección de Cambios Urbanos - Chaitén

Este notebook documenta los métodos de detección de cambios implementados.

## 1. Métodos Implementados

### Método 1: Diferencia de Índices (NDVI)
- Calcula: `NDVI_t2 - NDVI_t1`
- Clasifica: pérdida (-1), sin cambio (0), ganancia (+1)
- **Ventajas**: Simple, rápido, fácil interpretación
- **Desventajas**: Solo detecta cambios de vegetación

### Método 2: Clasificación Multiíndice
- Combina NDVI, NDBI y NDWI
- Clases: Urbanización, Pérdida/Ganancia Vegetación, Cambios Agua
- **Ventajas**: Distingue tipos de cambio
- **Desventajas**: Más parámetros a calibrar

In [ ]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from pathlib import Path

PROCESSED_DIR = Path('../data/processed')

## 2. Justificación de Umbrales

| Umbral | Valor | Justificación |
|--------|-------|---------------|
| `ndvi_veg` | 0.3 | Separa vegetación activa de suelo/urbano |
| `ndbi_urbano` | 0.0 | NDBI positivo = superficies impermeables |
| `cambio_min` | 0.15 | Cambios < 15% pueden ser ruido atmosférico |
| `umbral_ndwi` | 0.1 | NDWI > 0.1 indica presencia de agua |

## 3. Visualización de Cambios Detectados

In [ ]:
def visualizar_cambio_diferencia(ruta):
    """Visualiza el resultado del Método 1."""
    with rasterio.open(ruta) as src:
        cambio = src.read(1)
    
    cmap = ListedColormap(['red', 'gray', 'green'])
    
    fig, ax = plt.subplots(figsize=(10, 8))
    im = ax.imshow(cambio, cmap=cmap, vmin=-1, vmax=1)
    ax.set_title('Método 1: Diferencia de NDVI')
    
    cbar = plt.colorbar(im, ax=ax, ticks=[-1, 0, 1])
    cbar.ax.set_yticklabels(['Pérdida', 'Sin Cambio', 'Ganancia'])
    
    return fig

In [ ]:
def visualizar_cambio_clasificado(ruta):
    """Visualiza el resultado del Método 2."""
    with rasterio.open(ruta) as src:
        clase = src.read(1)
    
    colores = ['white', 'red', 'orange', 'green', 'blue', 'cyan']
    cmap = ListedColormap(colores)
    
    fig, ax = plt.subplots(figsize=(10, 8))
    im = ax.imshow(clase, cmap=cmap, vmin=0, vmax=5)
    ax.set_title('Método 2: Clasificación Multiíndice')
    
    cbar = plt.colorbar(im, ax=ax, ticks=[0, 1, 2, 3, 4, 5])
    cbar.ax.set_yticklabels(['Sin Cambio', 'Urbanización', 'Pérd. Veg', 
                            'Gan. Veg', 'Nuevo Agua', 'Pérd. Agua'])
    
    return fig

In [ ]:
# Visualizar resultados si existen
ruta_m1 = PROCESSED_DIR / 'cambio_diferencia_ndvi.tif'
ruta_m2 = PROCESSED_DIR / 'cambio_clasificado.tif'

if ruta_m1.exists():
    print("Método 1: Diferencia de Índices")
    visualizar_cambio_diferencia(ruta_m1)
    plt.show()
else:
    print(f"No existe: {ruta_m1}")

if ruta_m2.exists():
    print("\nMétodo 2: Clasificación Multiíndice")
    visualizar_cambio_clasificado(ruta_m2)
    plt.show()
else:
    print(f"No existe: {ruta_m2}")

## 4. Comparación de Métodos

| Aspecto | Método 1 (Diferencia) | Método 2 (Multiíndice) |
|---------|----------------------|------------------------|
| Complejidad | Baja | Media |
| Parámetros | 1 (umbral) | 4 (umbrales) |
| Clases | 3 | 6 |
| Tipo cambio | Solo vegetación | Múltiples |
| Velocidad | Muy rápida | Rápida |

## 5. Ejecutar Detección

```bash
python scripts/detect_changes.py
```

Genera:
- `cambio_diferencia_ndvi.tif` (Método 1)
- `cambio_clasificado.tif` (Método 2)
- `comparacion_metodos.md` (Documentación)

---
**Anterior**: [02_calculo_indices.ipynb](02_calculo_indices.ipynb) | **Siguiente**: [04_analisis_zonal.ipynb](04_analisis_zonal.ipynb)